In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import csv
import pandas as pd
import re
from scipy.stats import pearsonr
#import rpy2.robjects as robjects
import random
from statsmodels.stats.multitest import fdrcorrection
import copy
from collections import Counter
import seaborn as sns
from scipy.stats import binom_test
import gseapy as gs
from scipy.stats import ttest_ind
from scipy.stats import spearmanr

In [ ]:
#Ran this cell repeatedly with different parameters (e.g. uncommmenting folders, just high confidence ASD genes etc.)
#As needed for various figures
#Refer to this and the cell below throughout as "main cell"
sfari = pd.read_csv("SFARI-Gene_genes_03-28-2024release_05-09-2024export.csv")
sfari = sfari[sfari["gene-score"] == 1]
sfari = {"SFARI":list(sfari["gene-symbol"])}

SCZ = pd.read_csv("SCZ_Exome.csv")
SCZ = SCZ[SCZ["Q meta"] < 0.1]
psych = list(pd.read_csv("Psychosis_Genes.csv")["Gene"])
SCZ = {"SCZ":list(set(list(SCZ["Gene Symbol"])))}

#folder = "../../Allen_MTG_Single_Cell/Subclass/Processed"
folder = "../../Ma_Sestan_Single_Cell/Processed"
skel = "DESeq2_REPLACE_Human_Chimp.txt"
output = []
ind = []
mssng = pd.read_csv("MSSNG_Autism_2023_Cell.csv")
mssng_only_ptv = mssng[(mssng["De novo MisA"] < 1) & (mssng["De novo MisB"] < 1)]
out = []
out_der = []
out_der2 = []

if "Allen" in folder:
    celltypes = ['Sst', 'Sst Chodl', 'OPC', 'Micro-PVM', 'L6 CT', 'Lamp5_Lhx6', 'Chandelier', 'Pvalb', 'L4 IT', 'VLMC', 'L2/3 IT', 'Pax6', 'Endo', 'Vip', 'Astro', 'L5 IT', 'L5/6 NP', 'Lamp5', 'Sncg', 'L5 ET', 'Oligo', 'L6 IT Car3', 'L6 IT', 'L6b']
    celltypes = [x.replace(" ", "_").replace("/", "-") for x in celltypes]
else:
    celltypes = ['PVALB', 'SST_NPY', 'Astro', 'L5-6_NP', 'RB', 'VIP', 'Endo', 'SST_HGF', 'Immune', 'L6_CT', 'L6_IT-1', 'L3-5_IT-3', 'Micro', 'L6_IT-2', 'Oligo', 'L5_ET', 'L3-5_IT-1', 'ADARB2_KCNG1', 'PVALB_ChC', 'OPC', 'L3-5_IT-2', 'VLMC', 'SMC', 'L6B', 'SST', 'L2-3_IT', 'LAMP5_RELN', 'PC', 'LAMP5_LHX6']

#Compute binomial test p-values
def compute_ps(de, ct, scz = False):
    if scz:
        des = de.loc[np.intersect1d(SCZ["SCZ"], de.index)]
    elif neg_act:
        des = de.loc[np.intersect1d(activity["Negative"], de.index)]
    elif pos_act:
        des = de.loc[np.intersect1d(activity["Positive"], de.index)]
    elif mdown:
        des = de.loc[np.intersect1d(mon_down_d["Mon_Down"], de.index)]
    elif syngo:
        des = de.loc[np.intersect1d(syngo_d["syngo"], de.index)]
    else:
        des = de.loc[np.intersect1d(sfari["SFARI"], de.index)]
    de_mssng = de.loc[np.intersect1d(mssng["Gene"], de.index)]
    de_mssng_ptv = de.loc[np.intersect1d(mssng_only_ptv["Gene"], de.index)]
    
    up_human = de[de["log2FoldChange HumChp"] > 0].shape[0]
    down_human = de[de["log2FoldChange HumChp"] < 0].shape[0]
    up_human_sfari = des[des["log2FoldChange HumChp"] > 0].shape[0]
    down_human_sfari = des[des["log2FoldChange HumChp"] < 0].shape[0]
    up_human_mssng = de_mssng[de_mssng["log2FoldChange HumChp"] > 0].shape[0]
    down_human_mssng = de_mssng[de_mssng["log2FoldChange HumChp"] < 0].shape[0]
    up_human_mssng_ptv = de_mssng_ptv[de_mssng_ptv["log2FoldChange HumChp"] > 0].shape[0]
    down_human_mssng_ptv = de_mssng_ptv[de_mssng_ptv["log2FoldChange HumChp"] < 0].shape[0]
    
    p = binom_test(up_human_sfari, up_human_sfari + down_human_sfari, p = up_human/(up_human + down_human))
    res1 = [ct, up_human, down_human, up_human_sfari, down_human_sfari, up_human_mssng, down_human_mssng, up_human_mssng_ptv, down_human_mssng_ptv, p]
    
    de_sig = de[de["padj_mine HumChp"] < 0.05]
    if scz:
        de_sigs = de_sig.loc[np.intersect1d(SCZ["SCZ"], de_sig.index)]
    elif neg_act:
        de_sigs = de_sig.loc[np.intersect1d(activity["Negative"], de_sig.index)]
    elif pos_act:
        de_sigs = de_sig.loc[np.intersect1d(activity["Positive"], de_sig.index)]
    elif mdown:
        de_sigs = de_sig.loc[np.intersect1d(mon_down_d["Mon_Down"], de_sig.index)]
    elif syngo:
        de_sigs = de_sig.loc[np.intersect1d(syngo_d["syngo"], de_sig.index)]
    else:
        de_sigs = de_sig.loc[np.intersect1d(sfari["SFARI"], de_sig.index)]
    
    de_sig_mssng = de_sig.loc[np.intersect1d(mssng["Gene"], de_sig.index)]
    de_sig_mssng_ptv = de_sig.loc[np.intersect1d(mssng_only_ptv["Gene"], de_sig.index)]
    sig_up_human = de_sig[de_sig["log2FoldChange HumChp"] > 0].shape[0]
    sig_down_human = de_sig[de_sig["log2FoldChange HumChp"] < 0].shape[0]
    sig_up_human_sfari = de_sigs[de_sigs["log2FoldChange HumChp"] > 0].shape[0]
    sig_down_human_sfari = de_sigs[de_sigs["log2FoldChange HumChp"] < 0].shape[0]
    sig_up_human_mssng = de_sig_mssng[de_sig_mssng["log2FoldChange HumChp"] > 0].shape[0]
    sig_down_human_mssng = de_sig_mssng[de_sig_mssng["log2FoldChange HumChp"] < 0].shape[0]
    sig_up_human_mssng_ptv = de_sig_mssng_ptv[de_sig_mssng_ptv["log2FoldChange HumChp"] > 0].shape[0]
    sig_down_human_mssng_ptv = de_sig_mssng_ptv[de_sig_mssng_ptv["log2FoldChange HumChp"] < 0].shape[0]

    sig_p = binom_test(sig_up_human_sfari, sig_up_human_sfari + sig_down_human_sfari, p = sig_up_human/(sig_up_human + sig_down_human))
    
    res2 = [sig_up_human, sig_down_human, sig_up_human_sfari, sig_down_human_sfari, sig_up_human_mssng, sig_down_human_mssng, sig_up_human_mssng_ptv, sig_down_human_mssng_ptv, sig_p]
    return res1, res2

In [ ]:
scz = False
out = []
out_der = []
out_der2 = []
out3 = []

#Sometimes uncommented this so that the de was for L2/3 IT
#celltypes = ["L2-3_IT"]
for t in celltypes:
    de = pd.read_csv(folder + "/" + skel.replace("REPLACE", t), sep = "\t").set_index("Gene")
    if "Allen" in folder:
        de2 = pd.read_csv(folder + "/" + skel.replace("REPLACE", t).replace("Chimp", "Gorilla"), sep = "\t").set_index("Gene")
        de5 = pd.read_csv(folder + "/" + skel.replace("REPLACE", t).replace("Chimp", "Gorilla").replace("Human", "Chimp"), sep = "\t").set_index("Gene")
        de3 = pd.read_csv(folder + "/" + skel.replace("REPLACE", t).replace("Chimp", "Rhesus"), sep = "\t").set_index("Gene")
    de4 = pd.read_csv(folder + "/" + skel.replace("REPLACE", t).replace("Chimp", "Marmoset"), sep = "\t").set_index("Gene")

    de = de[["log2FoldChange", "padj_mine"]]
    de.columns = [x + " HumChp" for x in list(de.columns)]
    if "Allen" in folder:
        de2 = de2[["log2FoldChange", "padj_mine"]]
        de2.columns = [x + " HumGor" for x in list(de2.columns)]
        de = de.join(de2)
    de3 = de3[["log2FoldChange", "padj_mine"]]
    de3.columns = [x + " HumRhe" for x in list(de3.columns)]
    de4 = de4[["log2FoldChange", "padj_mine"]]
    de4.columns = [x + " HumMarm" for x in list(de4.columns)]
    de = de.join(de3)
    de = de.join(de4)
    if "Allen" in folder and not rhesus:
        de5 = de5[["log2FoldChange", "padj_mine"]]
        de5.columns = [x + " ChpGor" for x in list(de5.columns)]
        de = de.join(de5)
    de = de.dropna()

    if "Allen" in folder and not rhesus:
        derived = []
        for index, row in de.iterrows():
            if abs(row["log2FoldChange ChpGor"]) > abs(row["log2FoldChange HumChp"]) and abs(row["log2FoldChange HumGor"]) > abs(row["log2FoldChange HumChp"]):
                derived.append("Ambiguous")
            elif abs(row["log2FoldChange ChpGor"]) < abs(row["log2FoldChange HumGor"]):
                derived.append("Human derived")
            elif abs(row["log2FoldChange ChpGor"]) > abs(row["log2FoldChange HumGor"]):
                derived.append("Chimp derived")
        de["Derived"] = derived
    
    
    res1, res2 = compute_ps(de, t, scz = scz)
    if "Allen" in folder:
        de_sig = de[de["padj_mine HumChp"] < 0.05]
        if scz:
            des = de.loc[np.intersect1d(SCZ["SCZ"], de.index)]
            de_sigs = de_sig.loc[np.intersect1d(SCZ["SCZ"], de_sig.index)]
        elif neg_act:
            des = de.loc[np.intersect1d(activity["Negative"], de.index)]
            de_sigs = de_sig.loc[np.intersect1d(activity["Negative"], de_sig.index)]
        elif pos_act:
            des = de.loc[np.intersect1d(activity["Positive"], de.index)]
            de_sigs = de_sig.loc[np.intersect1d(activity["Positive"], de_sig.index)]
        elif mdown:
            print("AAA")
            des = de.loc[np.intersect1d(mon_down_d["Mon_Down"], de.index)]
            de_sigs = de_sig.loc[np.intersect1d(mon_down_d["Mon_Down"], de_sig.index)]
        else:
            des = de.loc[np.intersect1d(sfari["SFARI"], de.index)]
            de_sigs = de_sig.loc[np.intersect1d(sfari["SFARI"], de_sig.index)]
        total_hum_der = Counter(de["Derived"])["Human derived"]
        total_chp_der = Counter(de["Derived"])["Chimp derived"]
        sfari_hum_der = Counter(des["Derived"])["Human derived"]
        sfari_chp_der = Counter(des["Derived"])["Chimp derived"]
        p_der = binom_test(sfari_hum_der, sfari_hum_der + sfari_chp_der, p = total_hum_der/(total_hum_der + total_chp_der))
        
        
        sig_total_hum_der = Counter(de_sig["Derived"])["Human derived"]
        sig_total_chp_der = Counter(de_sig["Derived"])["Chimp derived"]
        sig_sfari_hum_der = Counter(de_sigs["Derived"])["Human derived"]
        sig_sfari_chp_der = Counter(de_sigs["Derived"])["Chimp derived"]
        sig_p_der = binom_test(sig_sfari_hum_der, sig_sfari_hum_der + sig_sfari_chp_der, p = sig_total_hum_der/(sig_total_hum_der + sig_total_chp_der))

        res3 = [t, total_hum_der, total_chp_der, sfari_hum_der, sfari_chp_der, p_der, sig_total_hum_der, sig_total_chp_der, sig_sfari_hum_der, sig_sfari_chp_der, sig_p_der]
        out_der.append(res3)
        de_hd = de[de["Derived"].isin(["Human derived"])]
        de_cd = de[de["Derived"].isin(["Chimp derived"])]

        res4, res5 = compute_ps(de_hd, t, scz = scz)
        out_der2.append(res4 + res5)
        sns.set(font_scale = 1.6)
    
    out.append(res1 + res2)
df = pd.DataFrame(out)
if "Allen" in folder:
    df_der = pd.DataFrame(out_der)
    df_der2 = pd.DataFrame(out_der2)

In [ ]:
#Make volcano plots for SCZ/ASD in MTG/DLPFC, need to change the title etc. 

df = pd.DataFrame(out)
cols = ["Cell_type", "up_human", "down_human", "up_human_sfari", "down_human_sfari", "up_human_mssng", "down_human_mssng", "up_human_mssng_ptv", "down_human_mssng_ptv", "binom_pval_sfari"]
cols = cols + [x + "_sig" for x in cols[1:]]
df.columns = cols
sns.set_style("white")
if not scz or "Allen" in folder or scz:
    print(True)
    df = df.sort_values("binom_pval_sfari_sig")
    df["Fold-enrichment"] = ((df["down_human_sfari_sig"] + 1)/(df["up_human_sfari_sig"] + 1))/(df["down_human_sig"]/df["up_human_sig"])
    df = df[(df["up_human_sig"] >= 500) | (df["down_human_sig"] >= 500)]
    df["FDR"] = fdrcorrection(df["binom_pval_sfari_sig"])[1]
    df["-log$_{10}$(FDR)"] = -np.log10(df["FDR"])
    sig = []
    for index, row in df.iterrows():
        if row["FDR"] < 0.05:
            sig.append("FDR < 0.05")
        else:
            sig.append("FDR > 0.05")
    df["Significant"] = sig
    palette_sig = {"FDR < 0.05":"red", "FDR > 0.05":"grey"}
    fig, ax = plt.subplots(figsize = (7, 5))
    sns.set(font_scale = 1.6)
    sns.set_style("white")
    sns.scatterplot(df, x = "Fold-enrichment", y = "-log$_{10}$(FDR)", hue = "Significant", palette = palette_sig, s = 60)
    plt.legend(frameon = True)
    plt.title("Bias toward lower expression of\nASD-linked genes in humans for DLPFC")
    plt.xlabel(None)
else:
    df = df.sort_values("binom_pval_sfari")
    df["Fold-enrichment"] = (df["down_human_sfari"]/df["up_human_sfari"])/(df["down_human"]/df["up_human"])
    df = df[~df["Cell_type"].isin(["Astro", "RB", "VLMC", "Micro", "Oligo", "OPC", "Endo", "Immune", "SMC", "PC"])]
    df["FDR"] = fdrcorrection(df["binom_pval_sfari_sig"])[1]
    df["-log$_{10}$(FDR)"] = -np.log10(df["FDR"])
    sig = []
    for index, row in df.iterrows():
        if row["FDR"] < 0.05:
            sig.append("FDR < 0.05")
        else:
            sig.append("FDR > 0.05")
    df["Significant"] = sig
    df = df.replace(np.inf, 10)
    palette_sig = {"FDR < 0.05":"red", "FDR > 0.05":"grey"}

    fig, ax = plt.subplots(figsize = (7, 5))
    sns.set(font_scale = 1.6)
    sns.set_style("white")
    sns.scatterplot(df, x = "Fold-enrichment", y = "-log$_{10}$(FDR)", hue = "Significant", palette = palette_sig, s = 1000)
    plt.legend(frameon = True)
    plt.title("Bias toward lower expression of\nASD-linked genes in humans")
    plt.xlabel(None)


In [ ]:
#Plotting the scz-linked genes with human-derived downregulation in MTG L2/3 IT neurons
de_scz = de.loc[np.intersect1d(SCZ["SCZ"], de.index)]
to_plot = de_scz[(de_scz["padj_mine HumChp"] < 0.05) & (de_scz["Derived"].isin(["Human derived"]))].sort_values("log2FoldChange HumChp")

out = []
for index, row in to_plot.iterrows():
    #out.append(["$_{" + index + "}$", row["log2FoldChange HumChp"], "Hybrid cortical organoid"])
    #out.append(["$_{" + index + "}$", row["log2FoldChange HumChp Adult"], "Adult MTG L2/3 IT"])
    if row["log2FoldChange HumChp"] < 0:
        x = "Down human"
    else:
        x = "Up human"
    out.append(["$_{" + index + "}$", row["log2FoldChange HumChp"], x])
to_plot = pd.DataFrame(out)
to_plot.columns = ["Gene", "Log$_{2}$ fold-change", "Direction"]
sns.set(font_scale = 1.5)
sns.set_style("white")
fig, ax = plt.subplots(figsize = (8, 3.5))
sns.set_style("white")
new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}
t_ax = sns.barplot(data = to_plot, x = "Gene", y = "Log$_{2}$ fold-change", hue = "Direction")


c = 0
for patch in t_ax.patches:
    if c < 5:
        patch.set_edgecolor(new_palette["Chimp"])
        patch.set_facecolor(new_palette["Chimp"] + "1A")
    else:
        patch.set_edgecolor(new_palette["Human"])
        patch.set_facecolor(new_palette["Human"] + "1A")
    c += 1
plt.title("Human-derived differential expression in MTG L2/3 IT neurons", size = 15)
plt.xticks(rotation=90)
plt.ylabel("Log$_{2}$ fold-change human/chimp", size = 14)
plt.legend([], [], frameon = False)

In [ ]:
#Plot up and down-regulated high-confidence ASD-linked genes in MTG L2/3 IT neurons 

new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}

sns.set(font_scale = 1.4)
sns.set_style("white")
fig, ax = plt.subplots(figsize=(3.5,5))
t_ax = sns.barplot({"Upregulated\nin human":25, "Downregulated\nin human":106}, errorbar=None, linewidth=2.5, edgecolor=".5", facecolor='#F2C91140', gap = 0.1, palette = {"Upregulated\nin human":new_palette["Human"], "Downregulated\nin human":new_palette["Chimp"]})
for patch in t_ax.patches:
    print(patch.get_x() + patch.get_width()/2)
c = 0
for patch in t_ax.patches:
    if c < 1:
        patch.set_edgecolor(new_palette["Human"])
        patch.set_facecolor(new_palette["Human"] + "1A")
    elif c == 1:
        patch.set_edgecolor(new_palette["Chimp"])
        patch.set_facecolor(new_palette["Chimp"] + "1A")
    c += 1
plt.ylabel("Number of genes")
plt.title("L2/3 IT neurons")

In [ ]:
#Plot up and down-regulated high-confidence ASD-linked genes in MTG L6 CT neurons 


new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}

sns.set(font_scale = 1.3)
sns.set_style("white")
fig, ax = plt.subplots(figsize=(3.5,5))
t_ax = sns.barplot({"Up-regulated\nin human":29, "Down-regulated\nin human":37}, errorbar=None, linewidth=2.5, edgecolor=".5", facecolor='#F2C91140', gap = 0.1, palette = {"Up-regulated\nin human":new_palette["Human"], "Down-regulated\nin human":new_palette["Chimp"]})
for patch in t_ax.patches:
    print(patch.get_x() + patch.get_width()/2)
c = 0
for patch in t_ax.patches:
    if c < 1:
        patch.set_edgecolor(new_palette["Human"])
        patch.set_facecolor(new_palette["Human"] + "1A")
    elif c == 1:
        patch.set_edgecolor(new_palette["Chimp"])
        patch.set_facecolor(new_palette["Chimp"] + "1A")
    c += 1
plt.ylabel("Number of genes")
plt.title("L6 CT neurons")

In [ ]:
#Plot up and down-regulated high-confidence ASD-linked genes in DLPFC L2/3 IT neurons 


new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}

sns.set(font_scale = 1.3)
sns.set_style("white")
fig, ax = plt.subplots(figsize=(3.5,5))
t_ax = sns.barplot({"Up-regulated\nin human":12, "Down-regulated\nin human":60}, errorbar=None, linewidth=2.5, edgecolor=".5", facecolor='#F2C91140', gap = 0.1, palette = {"Up-regulated\nin human":new_palette["Human"], "Down-regulated\nin human":new_palette["Chimp"]})
for patch in t_ax.patches:
    print(patch.get_x() + patch.get_width()/2)
c = 0
for patch in t_ax.patches:
    if c < 1:
        patch.set_edgecolor(new_palette["Human"])
        patch.set_facecolor(new_palette["Human"] + "1A")
    elif c == 1:
        patch.set_edgecolor(new_palette["Chimp"])
        patch.set_facecolor(new_palette["Chimp"] + "1A")
    c += 1
plt.ylabel("Number of genes")
plt.title("L2/3 IT neurons")

In [ ]:
#Plot up and down-regulated high-confidence ASD-linked genes in MTG L2/3 IT neurons 


new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}

sns.set(font_scale = 1.4)
sns.set_style("white")
fig, ax = plt.subplots(figsize=(3.5,5))
t_ax = sns.barplot({"Human":78, "Chimp":47}, errorbar=None, linewidth=2.5, edgecolor=".5", facecolor='#F2C91140', gap = 0.1, palette = new_palette)
for patch in t_ax.patches:
    print(patch.get_x() + patch.get_width()/2)
c = 0
for patch in t_ax.patches:
    if c < 1:
        patch.set_edgecolor(new_palette["Human"])
        patch.set_facecolor(new_palette["Human"] + "1A")
    elif c == 1:
        patch.set_edgecolor(new_palette["Chimp"])
        patch.set_facecolor(new_palette["Chimp"] + "1A")
    c += 1
plt.ylabel("Number of genes")
plt.title('High-confidence ASD-linked genes')
plt.xlabel("Lineage in which\nchange occurred")
plt.ylim(0, 91)
#plt.title("L2/3 IT neurons")

In [ ]:
#Run main cell with just L2/3 before generating this plot
sns.set(font_scale = 1.6)
sns.set_style("white")
fig, ax = plt.subplots(figsize = (7, 5))
sns.set_style("white")
new_palette = {"log$_{2}$(Human/Gorilla)":"#FF2C0C", "log$_{2}$(Chimp/Gorilla)":"#0058FF"}
des_plot = des[des["padj_mine HumChp"] < 0.05]
des_plot = des_plot[(np.abs(des_plot["log2FoldChange HumGor"]) < 3) & (np.abs(des_plot["log2FoldChange ChpGor"]) < 3)]
des_plot = des_plot[["log2FoldChange HumGor", "log2FoldChange ChpGor"]]
des_plot.columns = ["log$_{2}$(Human/Gorilla)", "log$_{2}$(Chimp/Gorilla)"]
sns.kdeplot(des_plot[["log$_{2}$(Human/Gorilla)", "log$_{2}$(Chimp/Gorilla)"]], fill = True, palette = new_palette)
plt.ylim(0, 0.65)
plt.vlines(x=0, ymin=0, ymax=0.65, color = "black", linestyles = "dashed")
plt.xlabel("Log$_{2}$ fold-change")
print(np.median(des_plot[des_plot["log$_{2}$(Human/Gorilla)"] < 0]["log$_{2}$(Human/Gorilla)"]))
print(np.median(des_plot[des_plot["log$_{2}$(Chimp/Gorilla)"] > 0]["log$_{2}$(Chimp/Gorilla)"]))
ax.legend([], [], frameon = False)
plt.title("LFC distribution for ASD-linked genes")
ttest_ind(np.abs(des_plot[des_plot["log$_{2}$(Human/Gorilla)"] < 0]["log$_{2}$(Human/Gorilla)"]), np.abs(des_plot[des_plot["log$_{2}$(Chimp/Gorilla)"] > 0]["log$_{2}$(Chimp/Gorilla)"]))

In [ ]:
#Checking how many high confidence ASD genes are high pLI
v = pd.read_csv("gnomad.v4.1.constraint_metrics.tsv", sep = "\t")
v = v.drop_duplicates("gene")
print("All genes: ", v[v["lof.pLI"] > 0.9].shape[0]/v.shape[0])

v = pd.read_csv("gnomad.v4.1.constraint_metrics.tsv", sep = "\t")
v = v[v["gene"].isin(sfari["SFARI"])]
v = v.drop_duplicates("gene")
print("SFARI genes: ", v[v["lof.pLI"] > 0.9].shape[0]/v.shape[0])

In [ ]:
#Sign test in cortical organoid hybrids

out = []
for boolean in [1, 0]:
    if boolean:
        sfari = pd.read_csv("SFARI-Gene_genes_03-28-2024release_05-09-2024export.csv")
        sfari = sfari[sfari["gene-score"] == 1]
        sfari = {"SFARI":list(sfari["gene-symbol"])}
    else:
        sfari = pd.read_csv("SFARI-Gene_genes_03-28-2024release_05-09-2024export.csv")
        sfari = {"SFARI":list(sfari["gene-symbol"])}
    for t in ["CS_D100", "CS_D150"]:
        v = pd.read_csv("Agoglia/Agoglia_Fraser_2021_DESeq2_HumChp_Hybrid_" + t + "_chpr.txt", sep = "\t").set_index("Gene").dropna()
        v.columns = [x + " HumChp" for x in v.columns]
        res1, res2 = compute_ps(v, "Agoglia " + str(t), scz = False, syngo=True)
        out.append(res1 + res2 + [boolean])
df = pd.DataFrame(out)
cols = ["Day", "up_human", "down_human", "up_human_sfari", "down_human_sfari", "up_human_mssng", "down_human_mssng", "up_human_mssng_ptv", "down_human_mssng_ptv", "binom_pval_sfari"]
cols = cols + [x + "_sig" for x in cols[1:]] + ["High confidence"]
df.columns = cols
df

In [ ]:
#Plotting bias in SCZ-linked genes for hybrid cortical organoids

sns.set(font_scale = 1.4)
sns.set_style("white")
fig, ax = plt.subplots(figsize=(3.5,5))

new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}
a = np.array([-18, -21])  # Positive values
b = np.array([12, 8])   # Negative values
cum_a = np.cumsum(a)
cum_b = np.cumsum(b)
import matplotlib.pyplot as plt

ind = np.arange(2)
p2 = plt.bar(ind, b, edgecolor = new_palette["Human"], facecolor = new_palette["Human"] + "1A")
p1 = plt.bar(ind, a, edgecolor = new_palette["Chimp"], facecolor = new_palette["Chimp"] + "1A")

plt.ylabel('Number of genes biased')
plt.title('SCZ-linked genes')
plt.xlabel("Human-chimp hybrid\ncortical organoids")
plt.xticks(ind, ['Day 100', 'Day 150'])
#plt.legend((p1[0], p2[0]), ('Lower expr. in human', 'Higher expr. in human'), bbox_to_anchor=(2.3, 1))
plt.yticks([-20, -10, 0, 10, 20], [20, 10, 0, 10, 20])
plt.ylim(-25, 25)
plt.show()

In [ ]:
#Plotting bias in high confidence ASD-linked genes for hybrid cortical organoids


sns.set(font_scale = 1.4)
sns.set_style("white")
fig, ax = plt.subplots(figsize=(3.5,5))

new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}
a = np.array([-21, -17])  # Positive values
b = np.array([8, 7])   # Negative values
cum_a = np.cumsum(a)
cum_b = np.cumsum(b)
import matplotlib.pyplot as plt

ind = np.arange(2)
p2 = plt.bar(ind, b, edgecolor = new_palette["Human"], facecolor = new_palette["Human"] + "1A")
p1 = plt.bar(ind, a, edgecolor = new_palette["Chimp"], facecolor = new_palette["Chimp"] + "1A")

plt.ylabel('Number of genes biased')
plt.title('High conf. ASD-linked genes')
plt.xlabel("Human-chimp hybrid\ncortical organoids")

plt.xticks(ind, ['Day 100', 'Day 150'])
#plt.legend((p1[0], p2[0]), ('Lower expr. in human', 'Higher expr. in human'), bbox_to_anchor=(2.3, 1))
plt.yticks([-20, -10, 0, 10, 20], [20, 10, 0, 10, 20])
plt.ylim(-25, 25)
plt.show()

In [ ]:
#Plotting bias in all ASD-linked genes for hybrid cortical organoids

sns.set(font_scale = 1.4)
sns.set_style("white")
fig, ax = plt.subplots(figsize=(3.5,5))

new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}
a = np.array([-120, -95])  # Positive values
b = np.array([85, 47])   # Negative values
cum_a = np.cumsum(a)
cum_b = np.cumsum(b)
import matplotlib.pyplot as plt

ind = np.arange(2)
p1 = plt.bar(ind, a, edgecolor = new_palette["Chimp"], facecolor = new_palette["Chimp"] + "1A")
p2 = plt.bar(ind, b, edgecolor = new_palette["Human"], facecolor = new_palette["Human"] + "1A")

plt.ylabel('Number of genes biased')
plt.title('ASD-linked genes')
plt.xticks(ind, ['Day 100', 'Day 150'])
plt.xlabel("Human-chimp hybrid\ncortical organoids")
plt.yticks([-125, -50, 0, 50, 125], [125, 50, 0, 50, 125])
plt.ylim(-150, 150)
#plt.legend((p1[0], p2[0]), ('Lower expr. in human', 'Higher expr. in human'), bbox_to_anchor=(1.1, 1))

plt.show()

In [ ]:
#Plotting bias in all ASD-linked genes, split by who is derived, for D150 hybrid cortical organoids

sns.set(font_scale = 1.4)
sns.set_style("white")
fig, ax = plt.subplots(figsize=(3.5,5))

new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}
a = np.array([-9, -20])  # Positive values
b = np.array([8, 2])   # Negative values
cum_a = np.cumsum(a)
cum_b = np.cumsum(b)
import matplotlib.pyplot as plt

ind = np.arange(2)
p1 = plt.bar(ind, a, edgecolor = new_palette["Chimp"], facecolor = new_palette["Chimp"] + "1A")
p2 = plt.bar(ind, b, edgecolor = new_palette["Human"], facecolor = new_palette["Human"] + "1A")

plt.ylabel(None)
plt.title('ASD-linked genes, D150')
plt.xticks(ind, ['Chimp', 'Human'])
plt.xlabel("Lineage in which\nchange occurred")
plt.yticks([-25, -10, 0, 10, 25], [25, 10, 0, 10, 25])
plt.ylim(-25, 25)
#plt.legend((p1[0], p2[0]), ('Lower expr. in human', 'Higher expr. in human'), bbox_to_anchor=(1.1, 1))

plt.show()

In [ ]:
#Plotting bias in all ASD-linked genes, split by who is derived, for D100 hybrid cortical organoids


sns.set(font_scale = 1.4)
sns.set_style("white")
fig, ax = plt.subplots(figsize=(3.5,5))

new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}
a = np.array([-9, -27])  # Positive values
b = np.array([10, 5])   # Negative values
cum_a = np.cumsum(a)
cum_b = np.cumsum(b)
import matplotlib.pyplot as plt

ind = np.arange(2)
p1 = plt.bar(ind, a, edgecolor = new_palette["Chimp"], facecolor = new_palette["Chimp"] + "1A")
p2 = plt.bar(ind, b, edgecolor = new_palette["Human"], facecolor = new_palette["Human"] + "1A")
plt.ylabel('Number of genes biased')

plt.title('All ASD-linked genes, D100')
plt.xticks(ind, ['Chimp', 'Human'])
plt.xlabel("Lineage in which\nchange occurred")
plt.yticks([-30, -15, 0, 15, 30], [30, 15, 0, 15, 30])
plt.ylim(-30, 30)
#plt.legend((p1[0], p2[0]), ('Lower expr. in human', 'Higher expr. in human'), bbox_to_anchor=(1.1, 1))

plt.show()

In [ ]:
#Plot of the normal distribution used to make Fig 6G

new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF", "Ancestral":"#804285"}
from scipy.stats import norm
mean = 8.0
std = 2.0
x_min = 0.0
x_max = 16.0
x = np.linspace(x_min, x_max, 100)
y = norm.pdf(x, mean, std)
plt.plot(x, y, color=new_palette["Ancestral"], fillstyle = "full")
plt.fill_between(x, y, color=new_palette["Ancestral"], alpha=0.4)
plt.xlim(x_min, x_max)
plt.ylim(0, 0.25)
plt.title('Normal Distribution Plot', fontsize=10)
plt.xlabel('x')
plt.ylabel('Probability Density')
plt.show()

In [ ]:
#Cell to do fisher_exact test can change FDR cutoff and t to do all the analysis in the paper

t = "CS_D150"
v = pd.read_csv("Agoglia/Agoglia_Fraser_2021_DESeq2_HumChp_Hybrid_" + t + "_chpr.txt", sep = "\t").set_index("Gene").dropna()
v.columns = [x + " HumChp" for x in v.columns]
sfari = pd.read_csv("SFARI-Gene_genes_03-28-2024release_05-09-2024export.csv")
#sfari = sfari[sfari["gene-score"] == 1]
sfari = {"SFARI":list(sfari["gene-symbol"])}

v_sig = v[v["padj_mine HumChp"] < 0.05]
v_sigs = v_sig.loc[np.intersect1d(v_sig.index, sfari["SFARI"])]

de_sig = de[de["padj_mine HumChp"] < 0.05]
de_sig = de_sig[['log2FoldChange HumChp', 'padj_mine HumChp', 'Derived']]
de_sig.columns = ['log2FoldChange HumChp Adult', 'padj_mine HumChp Adult', 'Derived']
v_sigs_joined = v_sigs.join(de_sig).dropna()
v_sigs_joined = v_sigs_joined[v_sigs_joined["Derived"].isin(["Human derived", "Chimp derived"])]
v_sigs_joined["bleh"] = list(np.sign(v_sigs_joined["log2FoldChange HumChp"]) == np.sign(v_sigs_joined["log2FoldChange HumChp Adult"]))
v_sigs_joined["bleh"] = v_sigs_joined["bleh"].replace(True, 1).replace(False, 0)
match_sign = np.sum(v_sigs_joined["bleh"])
total = v_sigs_joined.shape[0]
#Indicates that there is at least somewhat strong agreement
print(match_sign, total, binom_test(match_sign, total))

#Keep the ones with matching sign
v_sigs_joined = v_sigs_joined[v_sigs_joined["bleh"] == 1]
Counter(v_sigs_joined["Derived"])
down_human_humder = v_sigs_joined[(v_sigs_joined["Derived"].isin(["Human derived"])) & (v_sigs_joined["log2FoldChange HumChp"] < 0)].shape[0]
up_human_humder = v_sigs_joined[(v_sigs_joined["Derived"].isin(["Human derived"])) & (v_sigs_joined["log2FoldChange HumChp"] > 0)].shape[0]
down_human = v_sigs_joined[(v_sigs_joined["log2FoldChange HumChp"] < 0)].shape[0]
up_human = v_sigs_joined[(v_sigs_joined["log2FoldChange HumChp"] > 0)].shape[0]
print(binom_test(down_human_humder, (down_human_humder + up_human_humder)))
print(down_human_humder, up_human_humder, down_human, up_human)

down_human_chpder = v_sigs_joined[(v_sigs_joined["Derived"].isin(["Chimp derived"])) & (v_sigs_joined["log2FoldChange HumChp"] < 0)].shape[0]
up_human_chpder = v_sigs_joined[(v_sigs_joined["Derived"].isin(["Chimp derived"])) & (v_sigs_joined["log2FoldChange HumChp"] > 0)].shape[0]

#Strongly indicates that the selection is human-derived!
from scipy.stats import fisher_exact
print(fisher_exact([[down_human_humder, up_human_humder], [down_human_chpder, up_human_chpder]]))
print(down_human_humder, up_human_humder, down_human_chpder, up_human_chpder)
#List of possible candidate genes
print(list(v_sigs_joined[(v_sigs_joined["Derived"].isin(["Human derived"])) & (v_sigs_joined["log2FoldChange HumChp"] < 0)].index))

In [ ]:
#Plot human-derived ASE for ASD-linked genes in D150 cortical organoids

to_plot = v_sigs_joined[(v_sigs_joined["Derived"].isin(["Human derived"]))].sort_values("log2FoldChange HumChp")
out = []
for index, row in to_plot.iterrows():
    #out.append(["$_{" + index + "}$", row["log2FoldChange HumChp"], "Hybrid cortical organoid"])
    #out.append(["$_{" + index + "}$", row["log2FoldChange HumChp Adult"], "Adult MTG L2/3 IT"])
    if row["log2FoldChange HumChp"] < 0:
        x = "Down human"
    else:
        x = "Up human"
    out.append(["$_{" + index + "}$", row["log2FoldChange HumChp"], x])
to_plot = pd.DataFrame(out)
to_plot.columns = ["Gene", "Log$_{2}$ fold-change", "Direction"]
sns.set(font_scale = 1.5)
sns.set_style("white")
fig, ax = plt.subplots(figsize = (9, 4))
sns.set_style("white")
new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}
t_ax = sns.barplot(data = to_plot, x = "Gene", y = "Log$_{2}$ fold-change", hue = "Direction")


c = 0
for patch in t_ax.patches:
    if c < 20:
        patch.set_edgecolor(new_palette["Chimp"])
        patch.set_facecolor(new_palette["Chimp"] + "1A")
    else:
        patch.set_edgecolor(new_palette["Human"])
        patch.set_facecolor(new_palette["Human"] + "1A")
    c += 1
plt.title("Human-derived ASE in day 150 cortical organoids", size = 15)
plt.xticks(rotation=90)
plt.ylabel("Log$_{2}$ fold-change human/chimp", size = 14)
plt.legend([], [], frameon = False)